- Juan Camilo Salazar(A00381085)
- Samuel Gutierrez (A00381035)
- Camilo Carmona (A00381090)

# Prototype of the underlying expert system

##Imports

In [ ]:
!pip install pgmpy
!pip install experta
!pip install frozendict==2.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
from experta import *
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import pandas as pd

##Bayesian network

In [ ]:
dataAnxiety = pd.DataFrame(data={'p1':['1','2','3','0','1','3','2','1'],
                          'p2':['1','2','3','0','2','1','0','3'],
                          'p3':['1','2','3','0','1','3','3','2'],
                          'p4':['1','2','3','0','1','1','3','3'],
                          'p5':['1','2','3','0','2','0','2','3'],
                          'Anxiety':['mild','moderate','severe','none','mild','moderate','moderate','moderate']})

dataDepression = pd.DataFrame(data={'p1':['1','2','3','0','1'],
                          'p2':['1','2','3','0','2'],
                          'p3':['1','2','3','0','1'],
                          'p4':['1','2','3','0','1'],
                          'p5':['1','2','3','0','2'],
                          'Depression':['mild','moderate','moderate','mild','mild']})

In [ ]:
modelAnxiety = BayesianNetwork([('p1' , 'Anxiety' ) , ('p2' , 'Anxiety' ) , ('p3' , 'Anxiety' ) , ('p4' , 'Anxiety' ), ('p5' , 'Anxiety' )])

modelDepression = BayesianNetwork([('p1' , 'Depression' ) , ('p2' , 'Depression' ) , ('p3' , 'Depression' ) , ('p4' , 'Depression' ), ('p5' , 'Depression' )])

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(modelAnxiety, dataAnxiety)
def print_full (cpd):
  backup = TabularCPD._truncate_strtable
  TabularCPD._truncate_strtable = lambda self, x: x
  print (cpd)
  TabularCPD._truncate_strtable = backup

modelAnxiety.add_cpds(mle.estimate_cpd('p1'),mle.estimate_cpd('p2'),mle.estimate_cpd('p3'),mle.estimate_cpd('p4'),
               mle.estimate_cpd('p5'),mle.estimate_cpd('Anxiety'))

mle = MaximumLikelihoodEstimator(modelDepression, dataDepression)
def print_full (cpd):
  backup = TabularCPD._truncate_strtable
  TabularCPD._truncate_strtable = lambda self, x: x
  print (cpd)
  TabularCPD._truncate_strtable = backup

modelDepression.add_cpds(mle.estimate_cpd('p1'),mle.estimate_cpd('p2'),mle.estimate_cpd('p3'),mle.estimate_cpd('p4'),
               mle.estimate_cpd('p5'),mle.estimate_cpd('Depression') )

In [ ]:
from pgmpy.inference import VariableElimination
infer1 = VariableElimination (modelAnxiety)
post_p1 = infer1.query(["Anxiety"], evidence={"p1": "3","p2": "0", "p3": "3", "p4": "3", "p5": "0"})
print (post_p1)

infer2 = VariableElimination (modelDepression)
post_p2 = infer2.query(["Depression"], evidence={"p1": "3","p2": "3", "p3": "3", "p4": "3", "p5": "3"})
print (post_p2)

#print(post_p1.get_value(Anxiety="mild"))

+-------------------+----------------+
| Anxiety           |   phi(Anxiety) |
+===================+================+
| Anxiety(mild)     |         0.5000 |
+-------------------+----------------+
| Anxiety(moderate) |         0.5000 |
+-------------------+----------------+
+----------------------+-------------------+
| Depression           |   phi(Depression) |
+======================+===================+
| Depression(mild)     |            0.0000 |
+----------------------+-------------------+
| Depression(moderate) |            1.0000 |
+----------------------+-------------------+


In [ ]:
class ProtoMentalHealthHelper(KnowledgeEngine):

  @DefFacts()
  def _initial_ ( self ) :
    yield Fact(anxietyMildText="""\nMild anxiety disorder. which may require only watchful waiting""")
    yield Fact(anxietyModerateText="""\nModerate anxiety disorder. Active treatment probably warranted""")
    yield Fact(despressionMildText="""\nAnswers suggest mild depression which may require only watchful waiting and repeated test at followup.""")
    yield Fact(despressionModerateText="""\n Answers suggest moderate depression severity; patient should have a treatment plan ranging form counseling, followup, and/or pharmacotherapy.""")

    yield Fact(condition='anxiety', minAge=15, maxAge=24, risk ='high' )
    yield Fact(condition='anxiety', minAge=25, maxAge=40, risk ='mid' )
    yield Fact(condition='anxiety', minAge=41, maxAge=80, risk ='high' )

    yield Fact(condition='depression', minAge=15, maxAge=24, risk ='high' )
    yield Fact(condition='depression', minAge=25, maxAge=40, risk ='mid' )
    yield Fact(condition='depression', minAge=41, maxAge=80, risk ='high' )

  @Rule(Fact(anxiety1=MATCH.anxiety1),Fact(anxiety2=MATCH.anxiety2),Fact(anxiety3=MATCH.anxiety3),Fact(anxiety4=MATCH.anxiety4),Fact(anxiety5=MATCH.anxiety5))
  def anxiety(self,anxiety1,anxiety2,anxiety3, anxiety4, anxiety5):

    resultTable = infer1.query(["Anxiety"], evidence={"p1": anxiety1,"p2": anxiety2, "p3": anxiety3, "p4": anxiety4, "p5": anxiety5})

    mild = resultTable.get_value(Anxiety="mild")
    moderate = resultTable.get_value(Anxiety="moderate")

    self.declare(Fact(anxietyPreMild=mild))
    self.declare(Fact(anxietyPreModerate= moderate))


  @Rule(Fact(anxietyPreMild= MATCH.anxietyPreMild),Fact(anxietyPreModerate= MATCH.anxietyPreModerate),TEST(lambda anxietyPreModerate, anxietyPreMild: anxietyPreModerate >= anxietyPreMild))
  def anxietyDeclareModerate(self,anxietyPreModerate):
      self.declare(Fact(anxietyModerate=True, anxValue= anxietyPreModerate))


  @Rule(Fact(anxietyPreMild= MATCH.anxietyPreMild),Fact(anxietyPreModerate= MATCH.anxietyPreModerate),TEST(lambda anxietyPreModerate, anxietyPreMild: anxietyPreModerate < anxietyPreMild))
  def anxietyDeclareMild(self,anxietyPreMild):
      self.declare(Fact(anxietyMild=True, anxValue= anxietyPreMild))


  @Rule(Fact(depression1=MATCH.depression1),Fact(depression2=MATCH.depression2),Fact(depression3=MATCH.depression3),Fact(depression4=MATCH.depression4),Fact(depression5=MATCH.depression5))
  def depression(self,depression1,depression2,depression3, depression4, depression5):

    resultTable = infer2.query(["Depression"], evidence={"p1": depression1,"p2": depression2, "p3": depression3, "p4": depression4, "p5": depression5})

    mild = resultTable.get_value(Depression="mild")
    moderate = resultTable.get_value(Depression="moderate")

    self.declare(Fact(depressionPreMild=mild))
    self.declare(Fact(depressionPreModerate= moderate))


  @Rule(Fact(depressionPreMild= MATCH.depressionPreMild),Fact(depressionPreModerate= MATCH.depressionPreModerate),TEST(lambda depressionPreModerate, depressionPreMild: depressionPreModerate >= depressionPreMild))
  def depressionDeclareModerate(self,depressionPreModerate):
    self.declare(Fact(depressionModerate=True, depValue= depressionPreModerate))

  @Rule(Fact(depressionPreMild= MATCH.depressionPreMild),Fact(depressionPreModerate= MATCH.depressionPreModerate),TEST(lambda depressionPreModerate, depressionPreMild: depressionPreModerate < depressionPreMild))
  def depressionDeclareMild(self,depressionPreMild):
    self.declare(Fact(depressionMild=True, depValue= depressionPreMild))


  @Rule(Fact(depressionMild=True, depValue= MATCH.depValue),Fact(anxietyMild=True, anxValue= MATCH.anxValue),TEST(lambda depValue, anxValue: depValue >= anxValue), Fact(despressionMildText=MATCH.despressionMildText))
  def depressionMildPrint(self,despressionMildText):
    print(despressionMildText)

  @Rule(Fact(depressionMild=True, depValue= MATCH.depValue),Fact(anxietyMild=True, anxValue= MATCH.anxValue),TEST(lambda anxValue, depValue: anxValue > depValue), Fact(despressionMildText=MATCH.despressionMildText))
  def anxietyMildPrint(self,despressionMildText):
    print(despressionMildText)


  @Rule(Fact(depressionModerate=True), Fact(despressionModerateText=MATCH.despressionModerateText))
  def depressionPrint(self, despressionModerateText):
    print(despressionModerateText)

  @Rule(Fact(anxietyModerate=True), Fact(anxietyModerateText=MATCH.anxietyModerateText))
  def anxietyPrint(self, anxietyModerateText):
    print(anxietyModerateText)





In [ ]:
def run(anxiety1, anxiety2, anxiety3, anxiety4, anxiety5, depression1, depression2, depression3, depression4, depression5):

  #print(anxiety1, anxiety2, anxiety3, anxiety4, anxiety5, depression1, depression2, depression3, depression4, depression5)

  expert_system = ProtoMentalHealthHelper()
  expert_system.reset()

  expert_system.declare(Fact( anxiety1= anxiety1))
  expert_system.declare(Fact( anxiety2= anxiety2))
  expert_system.declare(Fact( anxiety3= anxiety3))
  expert_system.declare(Fact( anxiety4= anxiety4))
  expert_system.declare(Fact( anxiety5= anxiety5))

  expert_system.declare(Fact( depression1= depression1))
  expert_system.declare(Fact( depression2= depression2))
  expert_system.declare(Fact( depression3= depression3))
  expert_system.declare(Fact( depression4= depression4))
  expert_system.declare(Fact( depression5= depression5))

  #print (expert_system.facts)

  expert_system.run()

In [ ]:
def main():

  print ("Welcome to the Mental Health Helper Sysyem!")
  print ("how often have you been bothered by the following over the past 2 weeks?")
  print ("""
    Not at all = 0
    Several days = 1
    More than half the days = 2
    Nearly every day = 3
    """)

  anxiety1 = input ("Feeling nervous, anxious, or on edge (0,1,2,3)")
  anxiety2 = input ("Not being able to stop or control worrying (0,1,2,3)")
  anxiety3 = input ("Worrying too much about different things (0,1,2,3)")
  anxiety4 = input ("Trouble relaxing (0,1,2,3)")
  anxiety5 = input ("Being so restless that it's hard to sit still (0,1,2,3)")

  depression1 = input ("Little interest or pleasure in doing things?(0,1,2,3)")
  depression2 = input ("Feeling down, depressed, or hopeless? (0,1,2,3)")
  depression3 = input ("Trouble falling or staying asleep, or sleeping too much? (0,1,2,3)")
  depression4 = input ("Feeling tired or having little energy? (0,1,2,3)")
  depression5 = input ("Poor appetite or overeating? (0,1,2,3)")

  run(anxiety1, anxiety2, anxiety3, anxiety4, anxiety5, depression1, depression2, depression3, depression4, depression5)

if __name__ == "__main__":
  main()

Welcome to the Mental Health Helper Sysyem!
how often have you been bothered by the following over the past 2 weeks?

    Not at all = 0
    Several days = 1
    More than half the days = 2
    Nearly every day = 3
    
Feeling nervous, anxious, or on edge (0,1,2,3)3
Not being able to stop or control worrying (0,1,2,3)3
Worrying too much about different things (0,1,2,3)3
Trouble relaxing (0,1,2,3)3
Being so restless that it's hard to sit still (0,1,2,3)3
Little interest or pleasure in doing things?(0,1,2,3)0
Feeling down, depressed, or hopeless? (0,1,2,3)0
Trouble falling or staying asleep, or sleeping too much? (0,1,2,3)0
Feeling tired or having little energy? (0,1,2,3)0
Poor appetite or overeating? (0,1,2,3)0

Moderate anxiety disorder. Active treatment probably warranted


##Test

In [ ]:
run("1", "1", "1", "1", "1", "1","1", "1", "1", "1")


Answers suggest mild depression which may require only watchful waiting and repeated test at followup.


In [ ]:
run("3", "3", "3", "3", "3", "1","1", "1", "1", "1")


Moderate anxiety disorder. Active treatment probably warranted


In [ ]:
run("3", "3", "3", "3", "3", "3","3", "3", "3", "3")


 Answers suggest moderate depression severity; patient should have a treatment plan ranging form counseling, followup, and/or pharmacotherapy.

Moderate anxiety disorder. Active treatment probably warranted


In [ ]:
run("2", "0", "3", "1", "0", "0","2", "3", "3", "1")


 Answers suggest moderate depression severity; patient should have a treatment plan ranging form counseling, followup, and/or pharmacotherapy.

Moderate anxiety disorder. Active treatment probably warranted
